## Agreements

In [207]:
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [208]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
     users.append(agg['user__username'])   
users.remove("jmperez")
users = ["dvalado", "gdamill", "gclerici", "fdesanctis", "asilva"]

In [209]:


base_query = {
    "user__username__in": users
}
ArticleLabel.objects.filter(**base_query).count()

69

¿Cuántos etiquetamos todes?

In [210]:
import pandas as pd

pd.options.display.max_columns = 40


articles_labelled = {label.article_id for label in ArticleLabel.objects.filter(**base_query)}

df = pd.DataFrame({"name":users})
df.set_index("name", inplace=True)

for label in ArticleLabel.objects.filter(**base_query):
    username = label.user.username
    df.loc[username, label.article.title] = int(label.is_interesting)
    
df

,Escándalo en España: fallan los tests de coronavirus fabricados en China,El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,"Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina""",Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad”,Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”,Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Protestas y cortes en la Ciudad: piqueteros marcharon en la 9 de Julio a favor de la toma de tierras,Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño,"""Evolucionen"": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero",Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,Intentó violar a su novia y ella lo asesinó a escobazos,Coronavirus en Argentina: Cristina Kirchner vuelve de Cuba junto a su hija Florencia y permanecerá aislada por 14 días,La Villa 31 lleva más de una semana sin agua y se disparan los contagios: ya hay 133 casos positivos y una muerte por coronavirus,Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”,“El país está tomando todas las medidas para afrontar el coronavirus”
name,,,,,,,,,,,,,,,,,
dvalado,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
gdamill,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
gclerici,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
fdesanctis,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
asilva,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


## Agreements sobre odio

Ok, acá agarremos todos los labels sobre comentarios de todes

In [211]:
article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)

skipped_articles = [
    #Article.objects.get(title__contains="Thelma Fardin contó cómo sigue la causa contra Juan Darthés").id,
    #Article.objects.get(title__contains="Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria").id,
]

formatted_labels = []

df_comments = pd.DataFrame({"name": users})

df_comments.set_index("name", inplace=True)

for article_label in article_labels:
    username = article_label.user.username
    if article_label.article.id in skipped_articles:
        continue
    for comment_label in article_label.comment_labels.all():
        df_comments.loc[username, comment_label.comment_id] = comment_label.is_hateful

# Saco los que no estén etiquetados por todes
#df_comments = df_comments[df_comments.columns[df_comments.notna().all()]]

df_comments

,341401,341402,341403,341404,341405,341406,341407,341408,341409,341410,341411,341412,341413,341414,341415,341416,341417,341418,341419,341420,...,338643,338644,338645,338646,338647,338648,338649,338650,338651,338652,338653,338654,338655,338656,338657,338658,338659,338660,338661,338662
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
dvalado,True,True,False,False,True,True,False,True,False,False,False,False,True,False,True,False,True,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gdamill,True,True,False,False,True,True,False,True,False,False,False,False,True,False,True,False,False,False,True,True,...,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
gclerici,True,True,False,False,True,True,False,True,False,False,False,False,True,False,True,False,True,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fdesanctis,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
asilva,True,True,False,False,True,True,False,True,False,False,False,False,False,False,True,False,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
from nltk import agreement

def kappa_of(df, subset=None):
    if subset:
        coders = df.loc[subset]
    else:
        coders = df
    
    coders = coders[coders.columns[coders.notna().all()]]
    
    coders = coders.values
    formatted_labels = []
    for i in range(coders.shape[0]):
        # Acá las ponemos siguiendo el siguiente formato
        # [num etiquetador, num instancia, label]
        for j in range(coders.shape[1]):
            formatted_labels.append((i, j, coders[i, j]))


    ratingtask = agreement.AnnotationTask(data=formatted_labels)

    return ratingtask.kappa(), coders.shape[1]


for subset in [
    users,
    ]:
    name = ' '.join(subset)
    kappa, instances = kappa_of(df_comments, subset)
    print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")
    
print("\n\nPares\n\n")

for i in range(len(users)):
    for j in range(i+1, len(users)):
        subset = [users[i], users[j]]
        name = ' '.join(subset)
        kappa, instances = kappa_of(df_comments, subset)
        print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")


Fleiss K para dvalado gdamill gclerici fdesanctis asilva = 0.619 (#instancias = 563)


Pares


Fleiss K para dvalado gdamill                          = 0.717 (#instancias = 611)
Fleiss K para dvalado gclerici                         = 0.596 (#instancias = 563)
Fleiss K para dvalado fdesanctis                       = 0.638 (#instancias = 611)
Fleiss K para dvalado asilva                           = 0.612 (#instancias = 611)
Fleiss K para gdamill gclerici                         = 0.579 (#instancias = 563)
Fleiss K para gdamill fdesanctis                       = 0.649 (#instancias = 611)
Fleiss K para gdamill asilva                           = 0.674 (#instancias = 611)
Fleiss K para gclerici fdesanctis                      = 0.556 (#instancias = 563)
Fleiss K para gclerici asilva                          = 0.542 (#instancias = 563)
Fleiss K para fdesanctis asilva                        = 0.605 (#instancias = 611)


Podemos calcularlo con krippendorff ya que algunos saltearon

In [214]:
import krippendorff

alpha = krippendorff.alpha(df_comments.values.astype('float'))

print(f"Alpha para {' - '.join(users)}: {alpha:.3f}")

for i in range(len(users)):
    for j in range(i+1, len(users)):
        subset = [users[i], users[j]]
        name = ' '.join(subset)
        
        alpha = krippendorff.alpha(df_comments.loc[subset].values.astype('float'))
        print(f"Alpha para {name:<40} = {alpha:.3f}")



Alpha para dvalado - gdamill - gclerici - fdesanctis - asilva: 0.617
Alpha para dvalado gdamill                          = 0.718
Alpha para dvalado gclerici                         = 0.597
Alpha para dvalado fdesanctis                       = 0.638
Alpha para dvalado asilva                           = 0.608
Alpha para gdamill gclerici                         = 0.579
Alpha para gdamill fdesanctis                       = 0.649
Alpha para gdamill asilva                           = 0.672
Alpha para gclerici fdesanctis                      = 0.556
Alpha para gclerici asilva                          = 0.537
Alpha para fdesanctis asilva                        = 0.602


In [215]:
print("Cantidad de mensajes etiquetados como odiosos")
df_comments.sum(axis=1)

Cantidad de mensajes etiquetados como odiosos


name
dvalado       176.0
gdamill       182.0
gclerici      169.0
fdesanctis    165.0
asilva        120.0
dtype: float64

## Agreements sobre cada categoría

In [216]:
idx = pd.MultiIndex.from_product([CommentLabel.type_mapping, users],
                                names=["categoria", "etiquetador"])

df_labels = pd.DataFrame(index=idx)


In [217]:

article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)


for article_label in article_labels:
    username = article_label.user.username
    
    if article_label.article.id in skipped_articles:
        continue
        
    for comment_label in article_label.comment_labels.all():
        for name, field_name in CommentLabel.type_mapping.items():
            df_labels.loc[(name, username), comment_label.comment_id] = getattr(comment_label, field_name)

#df_labels

In [218]:
print("##", ' - '.join(users), '\n')

#users = ["mkondra", "jmperez"]

for category in CommentLabel.type_mapping:
    try:
        kappa, instances = kappa_of(df_labels.loc[category].loc[users])
        print(f"| {category:<12}   |  {kappa:.3f}")
    except ZeroDivisionError as e:
        print(f"No hay elementos para calcular Kappa de {category} - salteando")

print(f"Instancias: {instances}")


## dvalado - gdamill - gclerici - fdesanctis - asilva 

| MUJER          |  0.570
| LGBTI          |  0.725
| RACISMO        |  0.638
| POBREZA        |  0.611
| POLITICA       |  0.488
| DISCAPACIDAD   |  0.695
| ASPECTO        |  0.340
| CRIMINAL       |  0.764
No hay elementos para calcular Kappa de OTROS - salteando
Instancias: 563


## dvalado - gdamill - gclerici - fdesanctis 

| Anotador       | Kappa |
|----------------|-------|
| MUJER          |  0.583|
| LGBTI          |  0.740|
| RACISMO        |  0.644|
| POBREZA        |  0.613|
| POLITICA       |  0.482|
| DISCAPACIDAD   |  0.638|
| ASPECTO        |  0.514|
| CRIMINAL       |  0.740|

## dvalado - gdamill - gclerici 
| Anotador       |  Kappa| 
|----------------|-------|
| MUJER          |  0.575| 
| LGBTI          |  0.759| 
| RACISMO        |  0.737| 
| POBREZA        |  0.607| 
| POLITICA       |  0.564| 
| DISCAPACIDAD   |  0.570| 
| ASPECTO        |  0.610| 
| CRIMINAL       |  0.797| 
No hay elementos para calcular Kappa de OTROS - salteando

## Alpha Krippendorff

In [219]:
import krippendorff

users = list(df_labels.loc["MUJER"].index)

user_combinations = [users]
user_combinations += [[u1, u2] for u1, u2 in set((u1, u2) for u1 in users for u2 in users if u1 != u2 and u1 < u2)]

for user_list in user_combinations:
    print("="*80 + '\n' * 2)
    print(" - ".join(user_list), "\n")
    print("| Etiquetador    |        |")
    print("|----------------|--------|")
    for category in CommentLabel.type_mapping:
        mat = df_labels.loc[category].loc[user_list].values
        mat = mat.astype(float)
        alpha = krippendorff.alpha(mat)
        print(f"| {category:<12}   |  {alpha:.3f} |")





dvalado - gdamill - gclerici - fdesanctis - asilva 

| Etiquetador    |        |
|----------------|--------|
| MUJER          |  0.566 |
| LGBTI          |  0.728 |
| RACISMO        |  0.605 |
| POBREZA        |  0.619 |
| POLITICA       |  0.469 |
| DISCAPACIDAD   |  0.673 |
| ASPECTO        |  0.364 |
| CRIMINAL       |  0.759 |
| OTROS          |  nan |


dvalado - fdesanctis 

| Etiquetador    |        |
|----------------|--------|
| MUJER          |  0.656 |
| LGBTI          |  0.729 |
| RACISMO        |  0.457 |
| POBREZA        |  0.659 |
| POLITICA       |  0.424 |
| DISCAPACIDAD   |  0.888 |
| ASPECTO        |  0.527 |
| CRIMINAL       |  0.740 |
| OTROS          |  nan |


asilva - gdamill 

| Etiquetador    |        |
|----------------|--------|
| MUJER          |  0.545 |
| LGBTI          |  0.690 |
| RACISMO        |  0.658 |
| POBREZA        |  0.659 |
| POLITICA       |  0.559 |
| DISCAPACIDAD   |  0.711 |
| ASPECTO        |  0.197 |
| CRIMINAL       |  0.878 |
| OTROS

/home/jmperez/.local/share/virtualenvs/hatespeech-news-labelling-YFKq_jFG/lib/python3.8/site-packages/krippendorff/krippendorff.py:273: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (o * d).sum() / (e * d).sum()


| RACISMO        |  0.529 |
| POBREZA        |  0.660 |
| POLITICA       |  0.607 |
| DISCAPACIDAD   |  0.908 |
| ASPECTO        |  0.041 |
| CRIMINAL       |  0.701 |
| OTROS          |  nan |


gclerici - gdamill 

| Etiquetador    |        |
|----------------|--------|
| MUJER          |  0.554 |
| LGBTI          |  0.722 |
| RACISMO        |  0.723 |
| POBREZA        |  0.512 |
| POLITICA       |  0.468 |
| DISCAPACIDAD   |  0.519 |
| ASPECTO        |  0.503 |
| CRIMINAL       |  0.737 |
| OTROS          |  nan |


fdesanctis - gdamill 

| Etiquetador    |        |
|----------------|--------|
| MUJER          |  0.555 |
| LGBTI          |  0.717 |
| RACISMO        |  0.559 |
| POBREZA        |  0.659 |
| POLITICA       |  0.500 |
| DISCAPACIDAD   |  0.612 |
| ASPECTO        |  0.331 |
| CRIMINAL       |  0.729 |
| OTROS          |  nan |


fdesanctis - gclerici 

| Etiquetador    |        |
|----------------|--------|
| MUJER          |  0.561 |
| LGBTI          |  0.719 |
| RACISM



dvalado - gdamill - gclerici - fdesanctis - asilva 

| Clase          | Alpha  |
|----------------|--------|
| MUJER          |  0.566 |
| LGBTI          |  0.728 |
| RACISMO        |  0.605 |
| POBREZA        |  0.619 |
| POLITICA       |  0.469 |
| DISCAPACIDAD   |  0.673 |
| ASPECTO        |  0.364 |
| CRIMINAL       |  0.759 |
| OTROS          |  nan   |




asilva - fdesanctis - gdamill - lprats 

| Clase          | Acuerdo | 
|----------------|---------|
| MUJER          |  0.501  |
| LGBTI          |  0.772  |
| RACISMO        |  0.635  |
| POBREZA        |  0.499  |
| POLITICA       |  0.436  |
| DISCAPACIDAD   |  0.462  |
| ASPECTO        |  0.517  |
| CRIMINAL       |  0.931  |
| OTROS          |  -0.002 |


Apreciaciones generales:

- CRIMINAL: altísimo acuerdo
- MUJER: acuerdo razonable, pero hay que mejorar
- LGBTI y RACISMO: acuerdo relativamente alto
- POBREZA: ver qué pasa acá (ver Gabriel y Alan que etiquetaron lo mismo)
- POLITICA: esperable bajo acuerdo
- ASPECTO: ver acá qué pasó
- DISCAPACIDAD: ver también
- OTROS: ¿qué marcaron?




## MUJER


In [130]:
from collections import defaultdict

def no_null_columns(df):
    return df[df.columns[df.notna().all()]]

            
def show_from_df(df, show=False):
    article_to_comments = defaultdict(list)
    
    for comment_id in df.columns:
        comment = Comment.objects.get(id=comment_id)
        article_to_comments[comment.article_id].append(comment)
        
    for article_id, comments in article_to_comments.items():
        print("="*45 + "\n")
        article = comments[0].article
        print(article.title, "\n")
        
        for comment in sorted(comments, key=lambda c: sum(df[c.id])):
            print("")
            print("  - ",comment.text, end="")
            if show:
                yes = [annotator for annotator, v in df[comment.id].items() if v] 
                no = [annotator for annotator, v in df[comment.id].items() if not v] 
                print(f"\n     - SI: {yes}  NO: {no}")
            else:
                print("")

In [131]:
def get_agrees_and_disagrees(df_labels, category, users):    
    category_df = df_labels.loc[category].loc[users]

    category_df = no_null_columns(category_df)
    agree_comments = category_df[category_df.columns[category_df.all()]]
    non_hateful_comments = category_df[category_df.columns[(~category_df.astype(bool)).all()]]
    no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
    
    disagree_comments = category_df[category_df.columns[no_agree]]
    # Estos son muchos! Filtrar sólo los de aquellos artículos
    # donde haya habido algún acuerdo
    
    allowed_articles = set(
        c.article_id for c in Comment.objects.filter(id__in=agree_comments.columns)
    )
    
    allowed_comments = Comment.objects.filter(
        id__in=non_hateful_comments.columns,
        article_id__in=allowed_articles,
    )
    
    #print(len(allowed_comments))
    comment_ids = set(c.id for c in allowed_comments)
    
    non_hateful_comments = non_hateful_comments[[
        c for c in non_hateful_comments.columns if c in comment_ids
    ]]
    return agree_comments, disagree_comments, non_hateful_comments


agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "MUJER", users
)

In [132]:
show_from_df(agree_comments)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Hablo la catadora de pi........

Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 


  -  @usuario Terminala flaca, no existís. Te mandaste ese chamuyo para que te conociéramos y las aborteras te llevaran de bandera, pero tus 15 minutos de fama se acabaron. Proba con buscar un empleo honesto ahora, mamu. Aburriste! 😤

  -  @usuario No entiendo cómo le pueden creer si a cualquiera se le mete la pija para adentro con esta

  -  @usuario Esta pendeja usó a Juan una noche de tensión sexual y después es tan más agradecida en morderle la mano a quien le dio de comer.
Tal vez Juan no la comió bien. Pasa!

  -  @usuario Thelmita si sos petera deciloooo @usuario

"Evolucionen": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero 


  -  @usuario Lindo gatito

  -  @usuario Esa mentalidad de la mujer lati

Busquemos dónde hay discrepancias

In [133]:
show_from_df(disagree_comments, show=True)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario Y claro se llenó d comentarios de KKs a favor d la yegua mal parida y d culos rotos q defienden a otros culos rotos.. bueno o sea la yegua con la ley favoreció al país para q los maracas y lesbianas se casen, y eso la hace una gran persona, jajajaja mientras t chorea todo!!!
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Que la chupe
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Para decir algo público al menos deberia quitarse la máscara; así la reconocemos y sabemos seguro que fue ella quien lo dijo. Y no alguna nieta perdida, tal vez.
     - SI: ['gclerici']  NO: ['dvalado', 'gdamill', 'fdesanctis', 'asilva']

  -  @usuario Comerse a Fito es anti sanitario
     - SI: ['dvalado', 'gdamill']  NO: ['gclerici', 

In [142]:
%%capture
show_from_df(non_hateful_comments)

## LGBTI


In [143]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "LGBTI", users
)

In [144]:
show_from_df(agree_comments)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario asco

  -  @usuario 2 trolos casados por una rabina y escoltados por 2 pavorreales con la camiseta de river

  -  @usuario 🤢

  -  @usuario Aféitense, chicas! 💋

  -  @usuario ups el barba era trolis ... ups y parecía gente de bien

  -  @usuario Asco

  -  @usuario Asco

  -  @usuario La pucha, parece que son más los anormales. 
Hacia donde va la humanidad?
Que dos anormales se juntan, es noticia en un diario.

  -  @usuario Y a quién joraca le importa q estos 2 putitos se casen, encima se sacan selfies, q manga d trolos asquerosos culos rotos!! Como les gusta ventilar la mariconeada, se creen q uno está pendiente d sus vidas?!!


In [145]:
show_from_df(disagree_comments, show=True)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario Que cosa peluda 🦍😎
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario En ningun parrafo de los 70 libros que fueron escritos por inspiracion divina y en estos libros sagrados solamente figuran UNGIDOS los varones para el sacerdocio...!?
     - SI: ['dvalado']  NO: ['gdamill', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Completo
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario 😝q carajo nos importa
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Y que carajo me importa lo que hagan con sus C.
     - SI: ['gdamill', 'asilva']  NO: ['dvalado', 'gclerici', 'fdesanctis']

  -  @usuario Por estas cosas estamos como estamos😠
     - SI: ['gdamill', 'fdesanctis']  NO: ['dvalado', 'gclerici', 'asilva']

  -  @usuario Que raro , va contra la ley de Dios !!!! Que ra

In [146]:
%%capture
show_from_df(non_hateful_comments)

## RACISMO

In [147]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "RACISMO", users
)

In [148]:
show_from_df(agree_comments)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario Y la culp es de españa ?  O los putos chinos ?

  -  @usuario Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto

  -  @usuario CHINOS DE MIERDA
OJALÁ SE EXTINGAN

  -  @usuario Estos chinos son una mierda

Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos 


  -  @usuario Negra cloaquera

  -  @usuario tremenda negra boluda

  -  @usuario Negra y abortera


In [149]:
show_from_df(disagree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario Chinverguenzas
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario No podía fallar!  Jajajajaja Perdón que me ria pero jamás les hubiera comprado a ellos!
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Estos son socios del que hizo el virus!!!!
     - SI: ['dvalado', 'gclerici']  NO: ['gdamill', 'fdesanctis', 'asilva']

  -  @usuario China va a tener que pagarle al mindo todo el daño causado
     - SI: ['dvalado', 'gclerici']  NO: ['gdamill', 'fdesanctis', 'asilva']

  -  @usuario @usuario Pero como le vas a comprar los test a China??? A China no se le compra más nada.
*Se iba al chino a comprar vino
     - SI: ['gclerici', 'asilva']  NO: ['dvalado', 'gdamill', 'fdesanctis']

  -  @usuario Y son chinos cuando fueron fiables
     - SI: ['gdamill', 'gclerici']  NO: ['dvalado', 'fdesanctis', 'asilva']

  -  @usuario L

In [150]:
%%capture
show_from_df(non_hateful_comments)

## POBREZA

In [151]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POBREZA", users
)

In [152]:
show_from_df(agree_comments)


Protestas y cortes en la Ciudad: piqueteros marcharon en la 9 de Julio a favor de la toma de tierras 


  -  @usuario BALA

  -  @usuario VAGOS !! Usurpadores

  -  @usuario Bala y al ✨río✨

  -  @usuario Es el momento de tirar una bomba y que exploten todos


In [153]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus
     - SI: ['dvalado', 'gdamill', 'asilva']  NO: ['gclerici', 'fdesanctis']

Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 


  -  @usuario Pero GGG, las imágenes del conurbano, especialmente en La Matanza son del final absoluto de la cuarentena, todo el mundo en la calle, ferias de manteros, parece el inicio de temporada de saldos! ESO les tiene q preocupar!  Andar en bicicleta o correr de noche es un poco de salud!
     - SI: ['dvalado']  NO: ['gdamill', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Y si, al enano de Mierda le molestaba que los tipos salgan a correr, mientras los indios del congourbano están encerrados.
     - SI: ['dvalado', 'gdamill', 'asilva']  NO: ['gclerici', 'fdesanctis']

Protestas y cortes en la Ciudad: piquet

In [155]:
%%capture
show_from_df(non_hateful_comments)

## POLITICA

In [156]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POLITICA", users
)

In [157]:

show_from_df(agree_comments)


Protestas y cortes en la Ciudad: piqueteros marcharon en la 9 de Julio a favor de la toma de tierras 


  -  @usuario C5N el😺🇺🇾l 📩📩 💰💰la esta cubriendo y criticando por los contagios⁉️🤡🤡🤣🤣 RELATORES KUKA HDP

Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño 


  -  @usuario Que hermoso es boletear kukarachas

Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa 


  -  @usuario Pena de muerte deberían darle a ese enfermo mental comunista


In [158]:
show_from_df(disagree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario Pero que se puede esperar de comunistas y de asiaticos
     - SI: ['dvalado', 'gdamill', 'gclerici']  NO: ['fdesanctis', 'asilva']

  -  @usuario China te manda el virus
Te cobra el test...no anda
El partido comuns=ista china si sibrevife esto...nos gobernara a todos...
     - SI: ['dvalado', 'gdamill', 'gclerici']  NO: ['fdesanctis', 'asilva']

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario Y claro se llenó d comentarios de KKs a favor d la yegua mal parida y d culos rotos q defienden a otros culos rotos.. bueno o sea la yegua con la ley favoreció al país para q los maracas y lesbianas se casen, y eso la hace una gran persona, jajajaja mientras t chorea todo!!!
     - SI: ['gdamill', 'gclerici']  NO: ['dvalado', 'fdesanctis', 'asilva']

  -  @usuario Que suerte que el kirchnerismo legalizó que dos personas del mismo sexo se puedan casar. Fe

In [159]:
%%capture
show_from_df(non_hateful_comments)

## ASPECTO

In [160]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "ASPECTO", users
)

In [161]:
show_from_df(agree_comments)

In [163]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Pero miren a esta criatura de belcebú, con la cría de asesino en brazos para dar lástima. Ay! ay! Cobras y leones hambrientos" Problema solucionado.
     - SI: ['asilva']  NO: ['dvalado', 'gdamill', 'gclerici', 'fdesanctis']

  -  @usuario Un arma tatuada en la cara... Uno menos como dicen muchos...
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Delicatesen los tatus
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Podrían experimentar con estos seres posibles mutaciones y vacunas así hacen su aporte para la sociedad
     - SI: ['asilva']  NO: ['dvalado', 'gdamill', 'gclerici', 'fdesanctis']

  -  @usuario Mira lo que es ese muñeco por favor 💀
     - SI: ['gdamill', 'asilva']  NO: ['dvalado', 'gclerici', 'fdesanctis']

  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en cor

In [164]:
%%capture
show_from_df(non_hateful_comments)

## CRIMINAL

In [165]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "CRIMINAL", users
)

In [166]:
show_from_df(agree_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Uno menos.

  -  @usuario Al fin una buena!!!

  -  @usuario Uno menos..

  -  @usuario 👏🏻👏🏻👏🏻 sigan sigan

  -  @usuario Igual no servía para nada......

  -  @usuario Uno menos 💪🏼💪🏼💪🏼

  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus

  -  @usuario Loa beneficios del Covid19

  -  @usuario Deberían de aprovechar lo del coronavirus, y decir que atacó a todos los reclusos y, por eso debieron cremarlos🤷🏻‍♂️

  -  @usuario Pero miren a esta criatura de belcebú, con la cría de asesino en brazos para dar lástima. Ay! ay! Cobras y leones hambrientos" Problema solucionado.

  -  @usuario Una lacra menos 🤷🏼‍♂️

  -  @usuario Un arma tatuada en la cara... Uno menos como dicen muchos...

  -  @usuario Gracias Dios, te quedan vários todavía para llevarte.

  -  @usuario Uno menos 👏🏽👏🏽

  -  @usuario Uno menos

  -  @usuario Si se morían 1000 se solucio

In [167]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Delicatesen los tatus
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Bueno. Sigamos con cosas mas interesantes.
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Mira lo que es ese muñeco por favor 💀
     - SI: ['gdamill', 'asilva']  NO: ['dvalado', 'gclerici', 'fdesanctis']

  -  @usuario Q enorme perdida
     - SI: ['gdamill', 'fdesanctis']  NO: ['dvalado', 'gclerici', 'asilva']

  -  @usuario Punto a favor
     - SI: ['gdamill', 'fdesanctis', 'asilva']  NO: ['dvalado', 'gclerici']

  -  @usuario Pobre ciudadano, seguramente era inocente.
     - SI: ['dvalado', 'gdamill', 'fdesanctis']  NO: ['gclerici', 'asilva']

  -  @usuario Qepd ahre
     - SI: ['dvalado', 'gdamill', 'fdesanctis']  NO: ['gclerici', 'asilva']

  -  @usuario Que lastima
     - SI: ['dvalado', 'gdamill', 'fdesanctis']  NO: ['gclerici'

In [168]:
%%capture
show_from_df(non_hateful_comments, show=True)

## SALUD


In [169]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "DISCAPACIDAD", users
)

In [170]:
show_from_df(agree_comments)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Pasarte de merca durante años, es una verdadera irresponsabilidad sanitaria

  -  @usuario Irresponsable es q te sigas falopeando a los 60 años, vieja trola.

Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 


  -  @usuario Fardin deja la merca,te hace mal. 
le cagaste la vida a un tipo para tener cámara y te cagaste en las víctimas reales.

Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 


  -  @usuario Mala imagen sos vos viejo curda, el virus es difícil que llegue a la Argentina dijiste, te tomaste toda la merca que Toma Acsel hijo de un tren lleno de putas


In [171]:
show_from_df(disagree_comments, show=True)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario Tal vez después de eso se le pasa la histeria.
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Lleno d Nano Cerebros minusválidos infectados con el peor d todos los Virus e Peroncho KK. Ese cuando se les mete en el cuerpo hace estragos dejándolos inservibles y llenos d fanatismo, lo peor es q cada vez son + y así está el país!! Y no estsoy a favor d MM. Sociedad enferma!!!
     - SI: ['gclerici']  NO: ['dvalado', 'gdamill', 'fdesanctis', 'asilva']

Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 


  -  @usuario hundieron la económia, no tienen ni idea como manejar está gripe pero sí saben que si esto sale mal a la vista de todos, chau peronismo.
Nos van a confinar hasta la eternidad.
Hay q salir a recuperar la República, estamos en manos de subnormales
     - SI: ['gdamill'] 

In [172]:
%%capture
show_from_df(non_hateful_comments)

## OTROS

In [173]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "OTROS", users
)

In [174]:
show_from_df(agree_comments)

In [175]:
show_from_df(disagree_comments, show=True)

## LLAMADO A LA ACCIÓN

Queremos ver:

- Aquellas que hayan sido etiquetadas como odiosas


In [176]:
my_users = users
df_calls_for_action = pd.DataFrame({"name": my_users})
df_calls_for_action.set_index("name", inplace=True)

for comment_label in CommentLabel.objects.filter(is_hateful=True, article_label__user__username__in=my_users):
    username = comment_label.article_label.user.username
    df_calls_for_action.loc[username, comment_label.comment_id] = comment_label.calls_for_action

In [177]:
kappa_of(df_calls_for_action, users)

(0.6463562260389747, 72)

In [178]:
for i in range(len(df_calls_for_action)):
    user1 = df_calls_for_action.index[i]
    for j in range(i+1, len(df_calls_for_action)):
        user2 = df_calls_for_action.index[j]
        
        print(f"Kappa {user1:<10} - {user2:<10} = {kappa_of(df_calls_for_action, [user1, user2])}")

Kappa dvalado    - gdamill    = (0.6742301458670986, 134)
Kappa dvalado    - gclerici   = (0.4049586776859503, 120)
Kappa dvalado    - fdesanctis = (0.6077210460772103, 126)
Kappa dvalado    - asilva     = (0.8422330097087376, 104)
Kappa gdamill    - gclerici   = (0.49747899159663866, 115)
Kappa gdamill    - fdesanctis = (0.52102919346858, 121)
Kappa gdamill    - asilva     = (0.8659859604339504, 105)
Kappa gclerici   - fdesanctis = (0.5706214689265536, 114)
Kappa gclerici   - asilva     = (0.5854049719326385, 94)
Kappa fdesanctis - asilva     = (0.674342105263158, 99)


In [179]:
df_calls_for_action.sum(axis=1) / df_calls_for_action.notna().sum(axis=1)

name
dvalado       0.164773
gdamill       0.120879
gclerici      0.065089
fdesanctis    0.175758
asilva        0.183333
dtype: float64

In [180]:
df = no_null_columns(df_calls_for_action)

df.sum(axis=1)

name
dvalado       15.0
gdamill       17.0
gclerici       7.0
fdesanctis    12.0
asilva        15.0
dtype: float64

In [181]:

agree_comments = df[df.columns[df.all()]]
agree_comments.shape

no_agree = ~((df.sum() == 0) | (df.sum() == len(users)))
no_agree_comments = df[df.columns[no_agree]]

In [182]:
show_from_df(agree_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus

  -  @usuario Que no pare el fuego y rociarlo con nafta es una buena receta..

Protestas y cortes en la Ciudad: piqueteros marcharon en la 9 de Julio a favor de la toma de tierras 


  -  @usuario Bala y al ✨río✨

  -  @usuario Es el momento de tirar una bomba y que exploten todos


In [183]:
show_from_df(no_agree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario CHINOS DE MIERDA
OJALÁ SE EXTINGAN
     - SI: ['dvalado', 'gdamill', 'asilva']  NO: ['gclerici', 'fdesanctis']

El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Gracias Dios, te quedan vários todavía para llevarte.
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Si se morían 1000 se solucionaba el problema de raíz
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Parece que este virus nos va a depurar un poco de algunos males...
     - SI: ['gdamill']  NO: ['dvalado', 'gclerici', 'fdesanctis', 'asilva']

  -  @usuario Pero si tienen k morir todos los presos
     - SI: ['dvalado', 'gdamill', 'asilva']  NO: ['gclerici', 'fdesanctis']

  -  @usuario Ojala no sea el último
     - SI: ['dvalado', 'gdamill', 'asilva']  NO: ['gclerici', 'fdesanctis']

  -  @usuario Espere